Threat to Coral Reef from Fishing Practices	bio.024.4 http://www.wri.org/publication/reefs-risk-revisited

In [13]:
import numpy as np
import pandas as pd
import rasterio

import boto3
import requests as req

from matplotlib import pyplot as plt
%matplotlib inline
import os
import sys
import threading

If data already on s3, create a staging key and download to staging folder

In [14]:
s3_bucket = "wri-public-data"
s3_folder = "resourcewatch/bio_024_4_coral_reef_threat_from_fishing_practices/"
s3_key_orig = s3_folder + "bio_024_4_coral_reef_threat_from_fishing_practices.tif"
s3_key_edit = s3_key_orig[0:-4] + "_edit.tif"

temp_folder = "/Users/nathansuberi/Desktop/WRI_Programming/RW_Data/temp/"
local_orig = temp_folder + "bio_024_4.tif"
local_edit = local_orig[:-4] + "_edit.tif"

s3 = boto3.resource('s3')
s3.meta.client.download_file(s3_bucket, s3_key_orig, local_orig)
#s3.meta.client.download_file(s3_bucket, s3_key_edit, local_edit)

<b>Regardless of any needed edits, upload original file</b>

<i>Upload tif to S3 folder</i>

http://boto3.readthedocs.io/en/latest/guide/s3-example-creating-buckets.html

<i>Monitor Progress of Upload</i>

http://boto3.readthedocs.io/en/latest/_modules/boto3/s3/transfer.html
https://boto3.readthedocs.io/en/latest/guide/s3.html#using-the-transfer-manager

In [15]:
s3 = boto3.client("s3")

class ProgressPercentage(object):
        def __init__(self, filename):
            self._filename = filename
            self._size = float(os.path.getsize(filename))
            self._seen_so_far = 0
            self._lock = threading.Lock()

        def __call__(self, bytes_amount):
            # To simplify we'll assume this is hooked up
            # to a single filename.
            with self._lock:
                self._seen_so_far += bytes_amount
                percentage = (self._seen_so_far / self._size) * 100
                sys.stdout.write(
                    "\r%s  %s / %s  (%.2f%%)" % (
                        self._filename, self._seen_so_far, self._size,
                        percentage))
                sys.stdout.flush()

In [8]:
# Defined above:
# s3_bucket
# s3_key_orig
# s3_key_edit
# staging_key_orig
# staging_key_edit

s3.upload_file(local_key, s3_bucket, s3_key_orig,
                         Callback=ProgressPercentage(local_key))

/Users/nathansuberi/Desktop/WRI_Programming/RW_Data/Biodiversity/rf_ovf_adj1.tif  55644385 / 55644385.0  (100.00%)

Check for compression, projection

Create edit file if necessary

In [16]:
with rasterio.open(local_orig) as src:
    print(src.profile)

{'driver': 'GTiff', 'dtype': 'int16', 'nodata': -32768.0, 'width': 80150, 'height': 25456, 'count': 1, 'crs': CRS({'proj': 'cea', 'lon_0': -160, 'lat_ts': 0, 'x_0': 0, 'y_0': 0, 'datum': 'WGS84', 'units': 'm', 'no_defs': True}), 'transform': (-20037508.342787746, 500.0, 0.0, 6364114.668074458, 0.0, -500.0), 'affine': Affine(500.0, 0.0, -20037508.342787746,
       0.0, -500.0, 6364114.668074458), 'blockxsize': 128, 'blockysize': 128, 'tiled': True, 'compress': 'lzw', 'interleave': 'band'}


In [39]:
local_edit_tb = local_edit[:-4] + "_tb.tif"
local_edit_t = local_edit[:-4] + "_t.tif"
local_edit_b = local_edit[:-4] + "_b.tif"

with rasterio.open(local_orig) as src:
    
    data = src.read(1)
    
    # Return lat info
    south_lat = -90
    north_lat = 90
    # Return lon info
    west_lon = -180
    east_lon = 180
    # Transformation function
    transform = rasterio.transform.from_bounds(west_lon, south_lat, east_lon, north_lat, data.shape[1], data.shape[0])
    # Profile
    
    kwargs = src.profile
    kwargs.update(
        driver = 'GTiff', 
        dtype = rasterio.int16, 
        crs = 'EPSG:4326', 
        compress = 'lzw', 
        nodata = -9999,
        transform = transform,
    )
    kwargs_tiled_blocked = dict(kwargs)
    
    kwargs["tiled"] = False
    kwargs_blocked = dict(kwargs)
    
    kwargs.pop("blockxsize", None)
    kwargs.pop("blockysize", None)
    
    kwargs_no_tile_no_block = dict(kwargs)
    
    kwargs["tiled"] = True
    kwargs_tiled = dict(kwargs)
    
    np.putmask(data, data==-32768, -9999)

    with rasterio.open(local_edit_tb, 'w', **kwargs_tiled_blocked) as dst:
        dst.write(data.astype(kwargs['dtype']), 1)
    with rasterio.open(local_edit_t, 'w', **kwargs_tiled) as dst:
        dst.write(data.astype(kwargs['dtype']), 1)
    with rasterio.open(local_edit_b, 'w', **kwargs_blocked) as dst:
        dst.write(data.astype(kwargs['dtype']), 1)
    with rasterio.open(local_edit, 'w', **kwargs_no_tile_no_block) as dst:
        dst.write(data.astype(kwargs['dtype']), 1)
    


In [24]:
local_edit

'/Users/nathansuberi/Desktop/WRI_Programming/RW_Data/temp/bio_024_4_edit.tif'

In [41]:
with rasterio.open(local_edit) as src:
    print(src.profile)
    windows = src.block_windows()
    for ix, window in windows:
        print(window)
        break

{'driver': 'GTiff', 'dtype': 'int16', 'nodata': -9999.0, 'width': 80150, 'height': 25456, 'count': 1, 'crs': CRS({'init': 'epsg:4326'}), 'transform': (-180.0, 0.004491578290704928, 0.0, 90.0, 0.0, -0.007071024512884978), 'affine': Affine(0.004491578290704928, 0.0, -180.0,
       0.0, -0.007071024512884978, 90.0), 'compress': 'lzw', 'interleave': 'band', 'tiled': False}
((0, 1), (0, 80150))


In [42]:
with rasterio.open(local_edit_t) as src:
    print(src.profile)
    windows = src.block_windows()
    for ix, window in windows:
        print(window)
        break

{'driver': 'GTiff', 'dtype': 'int16', 'nodata': -9999.0, 'width': 80150, 'height': 25456, 'count': 1, 'crs': CRS({'init': 'epsg:4326'}), 'transform': (-180.0, 0.004491578290704928, 0.0, 90.0, 0.0, -0.007071024512884978), 'affine': Affine(0.004491578290704928, 0.0, -180.0,
       0.0, -0.007071024512884978, 90.0), 'compress': 'lzw', 'interleave': 'band', 'tiled': True, 'blockxsize': 256, 'blockysize': 256}
((0, 256), (0, 256))


In [43]:
with rasterio.open(local_edit_b) as src:
    print(src.profile)
    windows = src.block_windows()
    for ix, window in windows:
        print(window)
        break

{'driver': 'GTiff', 'dtype': 'int16', 'nodata': -9999.0, 'width': 80150, 'height': 25456, 'count': 1, 'crs': CRS({'init': 'epsg:4326'}), 'transform': (-180.0, 0.004491578290704928, 0.0, 90.0, 0.0, -0.007071024512884978), 'affine': Affine(0.004491578290704928, 0.0, -180.0,
       0.0, -0.007071024512884978, 90.0), 'blockxsize': '128', 'blockysize': '128', 'compress': 'lzw', 'interleave': 'band', 'tiled': False}
((0, 128), (0, 80150))


In [44]:
with rasterio.open(local_edit_tb) as src:
    print(src.profile)
    windows = src.block_windows()
    for ix, window in windows:
        print(window)
        break

{'driver': 'GTiff', 'dtype': 'int16', 'nodata': -9999.0, 'width': 80150, 'height': 25456, 'count': 1, 'crs': CRS({'init': 'epsg:4326'}), 'transform': (-180.0, 0.004491578290704928, 0.0, 90.0, 0.0, -0.007071024512884978), 'affine': Affine(0.004491578290704928, 0.0, -180.0,
       0.0, -0.007071024512884978, 90.0), 'blockxsize': 128, 'blockysize': 128, 'compress': 'lzw', 'interleave': 'band', 'tiled': True}
((0, 128), (0, 128))


Upload edited files to S3

In [40]:
# Defined above:
# s3_bucket
# s3_key_orig
# s3_key_edit
# staging_key_orig
# staging_key_edit

s3_key_edit_t = s3_key_edit[:-4] + "_t.tif"
s3_key_edit_b = s3_key_edit[:-4] + "_b.tif"
s3_key_edit_tb = s3_key_edit[:-4] + "_tb.tif"

s3.upload_file(local_edit, s3_bucket, s3_key_edit,
                         Callback=ProgressPercentage(local_edit))
s3.upload_file(local_edit_t, s3_bucket, s3_key_edit_t,
                         Callback=ProgressPercentage(local_edit_t))
s3.upload_file(local_edit_b, s3_bucket, s3_key_edit_b,
                         Callback=ProgressPercentage(local_edit_b))
s3.upload_file(local_edit_tb, s3_bucket, s3_key_edit_tb,
                         Callback=ProgressPercentage(local_edit_tb))

/Users/nathansuberi/Desktop/WRI_Programming/RW_Data/temp/bio_024_4_edit_tb.tif  54658774 / 54658774.0  (100.00%)

In [22]:
s3_key_edit

'resourcewatch/bio_024_4_coral_reef_threat_from_fishing_practices/bio_024_4_coral_reef_threat_from_fishing_practices_edit.tif'

Layer definition

https://github.com/resource-watch/notebooks/blob/master/ResourceWatch/Api_definition/layer_definition.ipynb

Upload to server destination

In [ ]:
# Too big for ArcGIS Online to upload using their web interface... 1 GB limit